In [10]:
x,y = 3,3
def cal_(x,y):
    if (x == 2)|(y == 2):
        return 3
    else:
        return x*y - (x-2)*(y-2)

In [22]:
x,y = 3,4
res = 0
for a in range(1,x-1):
    for b in range(1,y-1):
        # print(a,b)
        res+=2

res+=x-1+y-1

res

9

In [3]:
# load data.
def load_data():
    order = pd.read_excel(q1_ps + '/'+ os.listdir(q1_ps)[0])
    offer = pd.read_excel(q1_ps + '/'+ os.listdir(q1_ps)[1])
    offer = offer.rename({'Valid From':'valid_from', 'Valid To':'valid_to'}, axis = 1)
    order['ORDER_DATE'] = pd.to_datetime(order['ORDER_DATE'])
    offer['valid_from'] = pd.to_datetime(offer['valid_from'])
    offer['valid_to']   = pd.to_datetime(offer['valid_to'])
    return order, offer

order = load_data()[0]
offer = load_data()[1]

# explore data

In [4]:
# no non-ending data, all special offer have a due date.
offer['valid_to'].isna().sum(), order.shape

(0, (16069, 3))

In [5]:
# for some products, multi special offer observed
# just dups. not multi period of time.
# which makes primary key in offer table : Product_No_Start_With
# offer.groupby('Product_No_Start_With', as_index=False).count().sort_values('valid_from', ascending=False).head(7)
offer[offer["Product_No_Start_With"].isin(['7MF4635', 'C73451'])]

,Product_No_Start_With,valid_from,valid_to
46,7MF4635,2020-01-15,2021-11-30
47,7MF4635,2020-01-15,2021-11-30
48,7MF4635,2020-01-15,2021-11-30
96,C73451,2020-10-07,2021-11-30
97,C73451,2020-10-07,2021-11-30


In [6]:
# drop dups
offer = offer.drop_duplicates() #98 -> 95

In [7]:
# primary key in order table ['ORDER_NO', 'PROD_NO']
order.groupby(['ORDER_NO', 'PROD_NO'], as_index=False).count().sort_values('ORDER_DATE', ascending=False).head(5)

,ORDER_NO,PROD_NO,ORDER_DATE
0,SPRNO001031780X,7MF0300,1
10718,SPRNO1911271185,7MF4233,1
10705,SPRNO1911261155,7ML1118,1
10706,SPRNO1911261155,7ML5050,1
10707,SPRNO1911261155,7ML5430,1


In [8]:
# join key has multiple condition, check on it.
# see if special offer might be created twice in different length.
offer['key_len'] = offer['Product_No_Start_With'].apply(len)
offer['key'] = offer['Product_No_Start_With'].str.slice(0,5)
short_key = offer[offer['key_len'] < 7]['key']
offer[offer['key_len'] == 7][offer[offer['key_len'] == 7]['key'].isin(short_key)]

,Product_No_Start_With,valid_from,valid_to,key_len,key
42,7MF1572,2019-09-24,2021-11-30,7,7MF15


In [9]:
offer[offer['key'] == '7MF15']
# no special offer been created twice in different length.

,Product_No_Start_With,valid_from,valid_to,key_len,key
41,7MF156,2019-09-24,2021-11-30,6,7MF15
42,7MF1572,2019-09-24,2021-11-30,7,7MF15


# python solution

In [42]:
def answer_11(order , offer, lefton):
    """
    1. merge 3 times with regard to prod_no in different length.
    2. put flag on data by time restriction.
    3. save largest merged value.
    """
    offer = offer.drop_duplicates()
    order['prd6'] = order['PROD_NO'].str.slice(0,6)
    order['prd5'] = order['PROD_NO'].str.slice(0,5) 
    df = order.merge(offer, left_on=lefton, right_on='Product_No_Start_With', how = 'left')
    df['Flag'] = np.where((df['ORDER_DATE'] > df['valid_from']) & (df['ORDER_DATE'] < df['valid_to']), 1, 0) 
    return df[list(order.columns)+['Flag']]
    # return df
    
a = pd.DataFrame()
for i in ['PROD_NO', 'prd6', 'prd5']:
    a = a.append(answer_11(order , offer, i))
    
answer_11py = a.sort_values(
    [
        'Flag','ORDER_NO', 'PROD_NO', 'ORDER_DATE'
    ], ascending=False
    ).groupby(
        [
            'ORDER_NO', 'PROD_NO', 'ORDER_DATE'
        ], as_index=False
        ).first().drop(['prd6', 'prd5'], axis = 1)

In [43]:
answer_11py['Flag'].sum()

4910

> 4910

In [44]:
answer_11py

,ORDER_NO,PROD_NO,ORDER_DATE,Flag
0,SPRNO001031780X,7MF0300,2020-01-17 14:10:46,0
1,SPRNO001031780X,7MF0810,2020-01-17 14:10:46,0
2,SPRNO001031780X,7MF4033,2020-01-17 14:10:46,0
3,SPRNO001031780X,7MF4920,2020-01-17 14:10:46,1
4,SPRNO001031780X,7MF4980,2020-01-17 14:10:46,0
...,...,...,...,...
16064,SPRNO912061382X,7MF4433,2019-12-06 16:01:31,0
16065,SPRNO912061382X,7MF4923,2019-12-06 16:01:31,1
16066,SPRNO912061382X,A5E4914,2019-12-06 16:01:31,0
16067,SPRNO912271689X,7MF4635,2020-03-13 09:57:26,1


# SQL

In [6]:
from pandasql import sqldf 
order_df = order
pysqldf = lambda q: sqldf(q, globals())
sql = """
    SELECT 
    ORDER_NO, 
    PROD_NO, 
    ORDER_DATE,
    case when (
        b.Product_No_Start_With is not null
        )
    then 1
    else 0 end as Flag
    from order_df as a 
    left join
    (
        select *, row_number() over(
            partition by Product_No_Start_With, valid_from, valid_to
            ) as rn 
        from offer 
    ) as b
    on substr(a.PROD_NO, 1, length(b.Product_No_Start_With)) = b.Product_No_Start_With
    and b.rn = 1
    and (
        a.ORDER_DATE between b.valid_from and b.valid_to 
        or 
        b.valid_from is null
        )
"""

answer_11sql = pysqldf(sql)

answer_11sql['Flag'].sum()

> 4910

In [8]:
answer_11sql

,ORDER_NO,PROD_NO,ORDER_DATE,Flag
0,SPRNO1810170249,7ME6580,2018-10-17 11:02:14.000000,0
1,SPRNO1810170249,7ME6920,2018-10-17 11:02:14.000000,0
2,SPRNO1810170249,7MF4033,2018-10-17 11:02:14.000000,0
3,SPRNO1810170249,7MF4233,2018-10-17 11:02:14.000000,0
4,SPRNO1810170249,7MF4433,2018-10-17 11:02:14.000000,0
...,...,...,...,...
16064,SPRNO2011120701,7ML5430,2020-11-12 18:10:41.000000,1
16065,SPRNO2011120701,7ML5431,2020-11-12 18:10:41.000000,1
16066,SPRNO2011120701,7ML5830,2020-11-12 18:10:41.000000,0
16067,SPRNO2011130715,7ME4100,2020-11-13 12:20:13.000000,1
